In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time


In [3]:
from tkinter import Tk
from tkinter.filedialog import askopenfilename

Tk().withdraw()  # sembunyikan window tkinter
video_path = askopenfilename(
    title="Pilih File Video",
    filetypes=[
        ("Video Files", "*.mp4 *.avi *.mov *.mkv"),
        ("All Files", "*.*")
    ]
)

print("Video dipilih:", video_path)

Video dipilih: C:/Users/Lenovo/OneDrive/Pictures/Camera Roll/WIN_20251114_01_58_23_Pro.mp4


inisiasi face mesh

In [4]:
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

fungsi gaze ratio

In [5]:
def get_gaze_ratio(eye_points, facial_landmarks, frame_width, frame_height, is_left_eye=True):
    try:
        eye_left_point = (
            int(facial_landmarks[eye_points[0]].x * frame_width),
            int(facial_landmarks[eye_points[0]].y * frame_height)
        )
        eye_right_point = (
            int(facial_landmarks[eye_points[3]].x * frame_width),
            int(facial_landmarks[eye_points[3]].y * frame_height)
        )
        eye_top_point = (
            int(facial_landmarks[eye_points[1]].x * frame_width),
            int(facial_landmarks[eye_points[1]].y * frame_height)
        )
        eye_bottom_point = (
            int(facial_landmarks[eye_points[2]].x * frame_width),
            int(facial_landmarks[eye_points[2]].y * frame_height)
        )

        if is_left_eye:
            iris_indices = [468, 469, 470, 471]
        else:
            iris_indices = [473, 474, 475, 476]

        iris_x = np.mean([facial_landmarks[i].x * frame_width for i in iris_indices])
        iris_y = np.mean([facial_landmarks[i].y * frame_height for i in iris_indices])

        eye_width = abs(eye_right_point[0] - eye_left_point[0])
        eye_height = abs(eye_bottom_point[1] - eye_top_point[1])

        if eye_width == 0 or eye_height == 0:
            return "N/A", 0.5, 0.5

        horizontal_ratio = (iris_x - eye_left_point[0]) / (eye_width + 1e-6)
        vertical_ratio = (iris_y - eye_top_point[1]) / (eye_height + 1e-6)

        horizontal_ratio = np.clip(horizontal_ratio, 0, 1)
        vertical_ratio = np.clip(vertical_ratio, 0, 1)

        return "Fokus", horizontal_ratio, vertical_ratio

    except:
        return "N/A", 0.5, 0.5

Kalibrasi

In [6]:
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Tidak bisa membuka video.")

baseline_h, baseline_v = [], []
start_time = time.time()

print("Kalibrasi selama 3 detik...")

while time.time() - start_time < 3:
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_eye = [33, 160, 158, 133]
            right_eye = [362, 385, 387, 263]

            _, lh, lv = get_gaze_ratio(left_eye, face_landmarks.landmark, frame.shape[1], frame.shape[0], True)
            _, rh, rv = get_gaze_ratio(right_eye, face_landmarks.landmark, frame.shape[1], frame.shape[0], False)

            baseline_h.append((lh + rh) / 2)
            baseline_v.append((lv + rv) / 2)

base_h = np.median(baseline_h) if len(baseline_h) > 0 else 0.5
base_v = np.median(baseline_v) if len(baseline_v) > 0 else 0.5

print("Kalibrasi selesai!\n")

Kalibrasi selama 3 detik...
Kalibrasi selesai!



Analisis

In [7]:
gaze_data = {
    "Fokus": 0,
    "Melirik Kiri": 0,
    "Melirik Kanan": 0,
    "Melirik Atas": 0,
    "Melirik Bawah": 0,
    "N/A": 0
}

tolerance_h = 0.12
tolerance_v = 0.20
alpha = 0.25
prev_h, prev_v = base_h, base_v
frame_count = 0

cap = cv2.VideoCapture(video_path)

print("Mulai analisis video...\n")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)
    status = "N/A"

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_eye = [33, 160, 158, 133]
            right_eye = [362, 385, 387, 263]

            _, lh, lv = get_gaze_ratio(left_eye, face_landmarks.landmark, frame.shape[1], frame.shape[0])
            _, rh, rv = get_gaze_ratio(right_eye, face_landmarks.landmark, frame.shape[1], frame.shape[0])

            h_ratio = (lh + rh) / 2
            v_ratio = (lv + rv) / 2

            h_ratio = alpha * h_ratio + (1 - alpha) * prev_h
            v_ratio = alpha * v_ratio + (1 - alpha) * prev_v
            prev_h, prev_v = h_ratio, v_ratio

            if abs(h_ratio - base_h) < tolerance_h and abs(v_ratio - base_v) < tolerance_v:
                status = "Fokus"
            elif h_ratio - base_h > tolerance_h:
                status = "Melirik Kiri"
            elif base_h - h_ratio > tolerance_h:
                status = "Melirik Kanan"
            elif v_ratio - base_v > tolerance_v:
                status = "Melirik Bawah"
            elif base_v - v_ratio > tolerance_v:
                status = "Melirik Atas"

    gaze_data[status] += 1

    # === tampilkan frame dengan OpenCV, bukan matplotlib ===
    cv2.putText(frame, status, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                1, (0, 255, 0), 2)

    cv2.imshow("Deteksi Arah Pandangan", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print("Analisis selesai!\n")


Mulai analisis video...

Analisis selesai!



In [9]:
total_frames = sum(gaze_data.values())

fokus_frames = gaze_data["Fokus"]
cheat_frames = (
    gaze_data["Melirik Kiri"] +
    gaze_data["Melirik Kanan"] +
    gaze_data["Melirik Atas"] +
    gaze_data["Melirik Bawah"]
)

fokus_percent = (fokus_frames / total_frames) * 100
cheat_percent = (cheat_frames / total_frames) * 100

print("\n===== HASIL ANALISIS =====")
print(f"Total Frame: {total_frames}")
print(f"Fokus       : {fokus_frames} ({fokus_percent:.2f}%)")
print(f"Cheating    : {cheat_frames} ({cheat_percent:.2f}%)")
print("\nDetail Kategori:")
print(gaze_data)


===== HASIL ANALISIS =====
Total Frame: 410
Fokus       : 139 (33.90%)
Cheating    : 271 (66.10%)

Detail Kategori:
{'Fokus': 139, 'Melirik Kiri': 0, 'Melirik Kanan': 253, 'Melirik Atas': 18, 'Melirik Bawah': 0, 'N/A': 0}
